In [ ]:
# NOTE
# This notebook was executed and validated in Google Colab.
# AutoML (H2O, AutoGluon) is applied on the processed Titanic dataset.

import h2o
from h2o.automl import H2OAutoML

h2o.init()

train_h2o = h2o.H2OFrame(pd.concat([X_train, y_train], axis=1))
valid_h2o = h2o.H2OFrame(pd.concat([X_valid, y_valid], axis=1))

x_cols = X_train.columns.tolist()
y_col = "Survived"

train_h2o[y_col] = train_h2o[y_col].asfactor()
valid_h2o[y_col] = valid_h2o[y_col].asfactor()

aml = H2OAutoML(
    max_models=10,
    seed=42,
    balance_classes=True,
    sort_metric="AUC"
)
aml.train(x=x_cols, y=y_col, training_frame=train_h2o, leaderboard_frame=valid_h2o)

print(aml.leaderboard.head())
best_h2o = aml.leader
print("H2O valid AUC:", best_h2o.model_performance(valid_h2o).auc())

In [ ]:
lb_h2o = aml.leaderboard.as_data_frame()
display(lb_h2o.head(10))

In [ ]:
from autogluon.tabular import TabularPredictor

train_ag = pd.concat([X_train, y_train], axis=1)
valid_ag = pd.concat([X_valid, y_valid], axis=1)

predictor = TabularPredictor(label="Survived", eval_metric="roc_auc").fit(
    train_data=train_ag,
    time_limit=600,
    presets="medium_quality"
)

print(predictor.leaderboard(valid_ag, silent=True).head())

In [ ]:
lb = predictor.leaderboard(valid_ag, silent=True)
lb.head(10)

In [ ]:
from tpot import TPOTClassifier

tpot = TPOTClassifier()

try:
    tpot.fit(X_train, y_train)
    print("TPOT fit completed")
except Exception as e:
    print("TPOT execution issue due to API change:", e)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=300, random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)

y_proba = rf.predict_proba(X_valid)[:, 1]